In [23]:
import requests
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon as mplPolygon
from matplotlib.patches import Patch
import matplotlib.patches as mpatches
from cartopy.mpl.geoaxes import GeoAxes
import metpy
from metpy.io import parse_metar_file
from metpy.plots import USCOUNTIES
import pandas as pd

In [64]:
url = 'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=ACB&station=ADG&station=AMN&station=ANJ&station=APN&station=ARB&station=AZO&station=BAX&station=BEH&station=BIV&station=BTL&station=CAD&station=CFS&station=CIU&station=CMX&station=CVX&station=D95&station=DET&station=DRM&station=DTW&station=DUH&station=ERY&station=ESC&station=FFX&station=FKS&station=FNT&station=FPK&station=GLR&station=GOV&station=GRR&station=HAI&station=HTL&station=HYX&station=IKW&station=IMT&station=IRS&station=ISQ&station=IWD&station=JXN&station=JYM&station=LAN&station=LDM&station=LWA&station=MBL&station=MBS&station=MCD&station=MGN&station=MKG&station=MNM&station=MOP&station=MTC&station=OEB&station=ONZ&station=OSC&station=OZW&station=P53&station=P58&station=P59&station=P75&station=PHN&station=PLN&station=PTK&station=PZQ&station=RMY&station=RNP&station=RQB&station=SAW&station=SJX&station=SLH&station=TEW&station=TTF&station=TVC&station=VLL&station=Y31&station=Y70&station=YIP&data=all&year1=2022&month1=11&day1=18&year2=2022&month2=11&day2=19&tz=Etc%2FUTC&format=onlycomma&latlon=yes&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4'
df_parsed = pd.read_csv(url)
index_num = len(df_parsed.axes[0])
#keep = index_num - 74
#df_parsed = df.tail(302)
#df_parsed = df_parsed.drop_duplicates(subset=['station'])


wx_list = df_parsed['wxcodes'].unique()
snow_list = []
sum = 0

for i in wx_list:
    if 'SN' in i:
        snow_list.append(i)

# county_list = []
# for i in df.geometry:
#     county_list.append(i.get('coordinates'))
#     print(county_list)
station_list = ['LWA', 'AZO', 'BTL', 'RMY', 'FPK', 'Y70', 'GRR', 'RQB' , 'LDM', 'FFX', 'MKG', 'BIV']

snow_sum = 0
total = 0
for i in station_list:
    total_df = df_parsed[(df_parsed['station'] == i)]
    total += len(total_df)
    for j in snow_list:
        snow_df_parsed = df_parsed[(df_parsed['station'] == i) & (df_parsed['wxcodes'] == j)]
        #print(new_df_parsed)
        snow_sum += len(snow_df_parsed)

no_snow = total - snow_sum

print(snow_sum)
print(no_snow)
print((snow_sum/total)*100)
print((no_snow/total)*100)



776
161
82.81750266808965
17.18249733191035


In [60]:
url = 'https://mesonet.agron.iastate.edu/geojson/vtec_event.py?wfo=GRR&year=2022&phenomena=WS&etn=4&significance=W'
response = requests.get(url)
data = response.json()

events = data['features']
warning_data = []

for event in events:
    properties = event['properties']
    geometry = event['geometry']
    id = event['id']
    phenomena = properties['phenomena']
    eventid = properties['eventid']
    warning_data.append([id, phenomena, eventid, geometry])
df = pd.DataFrame(warning_data, columns=['id', 'phenomena', 'eventid', 'geometry'])

In [ ]:
for index, row in df.iterrows():
    geometry = row['geometry']
    
    if geometry:
        geom_type = geometry['type']
        coordinates = geometry['coordinates']
        
        if geom_type == 'MultiPolygon':
            for multi_coords in coordinates:
                for coords in multi_coords:
                    if isinstance(coords[0][0], float):
                        polygon = mplPolygon(coords, closed=True)